In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from itertools import product

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import pandas as pd
import seaborn as sns

import sbibm
from sbibm.utils.io import get_tensor_from_csv

import sbi.analysis as analysis

from notebook import nre_gamma_default, prepare_df_for_plot, get_metrics, wide_to_long

In [3]:
df = get_metrics(
    [
        "metrics-sbibm.csv",
    ],
    "sbibm",
)

df["gamma"] = df["gamma"].fillna(nre_gamma_default)

print(len(df))
print(df.columns)
print(df["algorithm"].unique())
print(df["task"].unique())
print(df["num_contrastive_parameters"].unique())
df.head()

127
Index(['task', 'seed', 'algorithm', 'K', 'num_atoms',
       'num_contrastive_parameters', 'gamma', 'extra_theta_factor', 'reuse',
       'hidden_features', 'num_blocks', 'AVG_LOG_RATIO', 'RT', 'C2ST_Z-01',
       'MMD-01', 'MEDDIST-01', 'NLTP-01', 'C2ST_Z-02', 'MMD-02', 'MEDDIST-02',
       'NLTP-02', 'C2ST_Z-03', 'MMD-03', 'MEDDIST-03', 'NLTP-03', 'C2ST_Z-04',
       'MMD-04', 'MEDDIST-04', 'NLTP-04', 'C2ST_Z-05', 'MMD-05', 'MEDDIST-05',
       'NLTP-05', 'C2ST_Z-06', 'MMD-06', 'MEDDIST-06', 'NLTP-06', 'C2ST_Z-07',
       'MMD-07', 'MEDDIST-07', 'NLTP-07', 'C2ST_Z-08', 'MMD-08', 'MEDDIST-08',
       'NLTP-08', 'C2ST_Z-09', 'MMD-09', 'MEDDIST-09', 'NLTP-09', 'C2ST_Z-10',
       'MMD-10', 'MEDDIST-10', 'NLTP-10', 'NLTP', 'num_simulations_simulator',
       'path', 'folder', 'expr'],
      dtype='object')
['CNRE']
['gaussian_linear' 'slcp' 'bernoulli_glm_raw' 'gaussian_linear_uniform'
 'bernoulli_glm' 'slcp_distractors' 'two_moons' 'gaussian_mixture' 'sir'
 'lotka_volterra']
[100]


,task,seed,algorithm,K,num_atoms,num_contrastive_parameters,gamma,extra_theta_factor,reuse,hidden_features,...,NLTP-09,C2ST_Z-10,MMD-10,MEDDIST-10,NLTP-10,NLTP,num_simulations_simulator,path,folder,expr
0,gaussian_linear,2819723463,CNRE,99,99,100,1.0,0,False,128,...,NaN,0.72725,0.053858,1.733452,NaN,NaN,1000.0,/home/ben/sci/cnre/infinite/sbibm/10-09-19/15,15,sbibm
1,slcp,284602413,CNRE,99,99,100,1.0,0,False,128,...,NaN,0.78335,0.044625,1.941932,NaN,NaN,100000.0,/home/ben/sci/cnre/infinite/sbibm/10-09-19/5,5,sbibm
2,bernoulli_glm_raw,22242810,CNRE,99,99,100,1.0,0,False,128,...,NaN,0.81815,0.142406,5.385165,NaN,NaN,10000.0,/home/ben/sci/cnre/infinite/sbibm/10-09-19/22,22,sbibm
3,gaussian_linear_uniform,1427711303,CNRE,99,99,100,1.0,0,False,128,...,NaN,0.83120,0.086919,1.637580,NaN,NaN,1000.0,/home/ben/sci/cnre/infinite/sbibm/10-09-19/9,9,sbibm
4,gaussian_linear_uniform,1502327203,CNRE,99,99,100,1.0,0,False,128,...,NaN,0.65645,0.028511,1.439581,NaN,NaN,10000.0,/home/ben/sci/cnre/infinite/sbibm/10-09-19/10,10,sbibm


In [4]:
df_wide = pd.wide_to_long(
    df,
    ["C2ST_Z"],
    i=[
        "task",
        "algorithm",
        "num_contrastive_parameters",
        "gamma",
        "num_blocks",
        "hidden_features",
        "seed",
    ],
    j="num_observation",
    sep="-",
)

# groupby drops NaNs
grp = df_wide.groupby(
    [
        "task",
        "algorithm",
        "num_simulations_simulator"
    ]
)
c2st_by_task_num_sim = grp.mean()["C2ST_Z"]

In [5]:
out = c2st_by_task_num_sim.groupby("num_simulations_simulator").mean()
out = out.to_frame().reset_index()
out

,num_simulations_simulator,C2ST_Z
0,1000.0,0.854154
1,10000.0,0.757458
2,100000.0,0.687906


In [6]:
from_sims_to_sym = {1000.0: "10³", 10000.0: "10⁴", 100000.0: "10⁵"}
out["num_simulations"] = out["num_simulations_simulator"].map(from_sims_to_sym)
out["C2ST"] = out["C2ST_Z"]
out["algorithm"] = "NRE-C"

## load sbibm results

In [7]:
mp = pd.read_csv("../remote/results/benchmarking_sbi/results/main_paper.csv")
results = mp.groupby(["algorithm", "num_simulations"]).mean().reset_index()

In [8]:
cat = pd.concat(
    [results[["algorithm", "num_simulations", "C2ST"]], out[["algorithm", "num_simulations", "C2ST"]]], 
    ignore_index=True
)

cat["algorithm"] = cat["algorithm"].map(lambda x: "\\" + "".join(x.split("-")))
cat["algorithm"] = cat["algorithm"].map(lambda x: "\\NREC (ours)" if x == "\\NREC" else x)

cat["Simulation budget"] = cat["num_simulations"]
cat = cat.drop("num_simulations", axis=1)
from_sym_to_sims = {"10³": r"$10^3$", "10⁴": r"$10^4$", "10⁵": r"$10^5$"}
cat["Simulation budget"] = cat["Simulation budget"].map(from_sym_to_sims)

cat["Algorithm"] = cat["algorithm"]
cat = cat.drop("algorithm", axis=1)

In [9]:
table = cat.groupby(["Algorithm", "Simulation budget"]).first().unstack()

order = [
    '\\NREC (ours)',
    '\\REJABC', 
    '\\NLE', 
    '\\NPE', 
    '\\NRE',  
    '\\SMCABC', 
    '\\SNLE', 
    '\\SNPE',
    '\\SNRE',
]
table = table.loc[order]

print(
    table.style.format(precision=3).to_latex(
        position_float="centering",
        hrules=True,
        caption="Simulation-based inference benchmark results.",
        label="table:sbibm"
    )
)

\begin{table}
\centering
\caption{Simulation-based inference benchmark results.}
\label{table:sbibm}
\begin{tabular}{lrrr}
\toprule
 & \multicolumn{3}{r}{C2ST} \\
Simulation budget & $10^3$ & $10^4$ & $10^5$ \\
Algorithm &  &  &  \\
\midrule
\NREC (ours) & 0.854 & 0.757 & 0.688 \\
\REJABC & 0.965 & 0.920 & 0.871 \\
\NLE & 0.826 & 0.753 & 0.723 \\
\NPE & 0.838 & 0.736 & 0.654 \\
\NRE & 0.867 & 0.811 & 0.762 \\
\SMCABC & 0.948 & 0.873 & 0.816 \\
\SNLE & 0.783 & 0.704 & 0.655 \\
\SNPE & 0.796 & 0.677 & 0.615 \\
\SNRE & 0.788 & 0.703 & 0.610 \\
\bottomrule
\end{tabular}
\end{table}

